In [8]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set random seed for reproducibility
tf.random.set_seed(42)

# Define constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 200
EPOCHS = 25

# Load pre-trained ResNet50 without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a final softmax layer
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze layers in the base ResNet50 model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/15ImagesinEachClassCubdataset/train',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/15ImagesinEachClassCubdataset/validation',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE)

# Evaluate accuracy
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/15ImagesinEachClassCubdataset/test',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)

Found 2000 images belonging to 200 classes.
Found 400 images belonging to 200 classes.
Epoch 1/25
62/62 [==============================] - 298s 5s/step - loss: 5.4386 - accuracy: 0.0025 - val_loss: 5.2995 - val_accuracy: 0.0052
Epoch 2/25
62/62 [==============================] - 331s 5s/step - loss: 5.3003 - accuracy: 0.0036 - val_loss: 5.2984 - val_accuracy: 0.0078
Epoch 3/25
62/62 [==============================] - 298s 5s/step - loss: 5.3003 - accuracy: 0.0036 - val_loss: 5.2983 - val_accuracy: 0.0026
Epoch 4/25
62/62 [==============================] - 331s 5s/step - loss: 5.2992 - accuracy: 0.0025 - val_loss: 5.2983 - val_accuracy: 0.0026
Epoch 5/25
62/62 [==============================] - 296s 5s/step - loss: 5.3002 - accuracy: 0.0015 - val_loss: 5.2970 - val_accuracy: 0.0078
Epoch 6/25
62/62 [==============================] - 331s 5s/step - loss: 5.2983 - accuracy: 0.0030 - val_loss: 5.2955 - val_accuracy: 0.0052
Epoch 7/25
62/62 [==============================] - 331s 5s/step - 

at 1 epochs testing accuracy is 0.005%.

at 10 epochs testing accuracy is 0.006%

In [9]:
accuracy = model.evaluate(test_generator)[1]
print("Test Accuracy:", accuracy)

19/19 [==============================] - 73s 4s/step - loss: 5.2589 - accuracy: 0.0067
Test Accuracy: 0.006677796132862568
